In [17]:
import tensorflow_hub as hub
module = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/3")


In [24]:
!pip install tensorflow_text

     |████████████████████████████████| 2.8 MB 2.8 MB/s eta 0:00:01
  Using cached tensorflow-2.4.1-cp37-cp37m-macosx_10_11_x86_64.whl (173.9 MB)
  Using cached tensorflow_estimator-2.4.0-py2.py3-none-any.whl (462 kB)
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached tensorboard-2.4.1-py3-none-any.whl (10.6 MB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Attempting uninstall: gast
    Found existing installation: gast 0.2.2
    Uninstalling gast-0.2.2:
      Successfully uninstalled gast-0.2.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successf

In [25]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

# Some texts of different lengths.
english_sentences = ["dog", "Puppies are nice.", "I enjoy taking long walks along the beach with my dog."]
italian_sentences = ["cane", "I cuccioli sono carini.", "Mi piace fare lunghe passeggiate lungo la spiaggia con il mio cane."]
japanese_sentences = ["犬", "子犬はいいです", "私は犬と一緒にビーチを散歩するのが好きです"]

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

# Compute embeddings.
en_result = embed(english_sentences)

In [26]:
en_result

<tf.Tensor: shape=(3, 512), dtype=float32, numpy=
array([[-0.00525213, -0.038312  , -0.00922013, ...,  0.0217905 ,
         0.06280179, -0.01622537],
       [-0.02267583, -0.06907186,  0.01550871, ...,  0.08762303,
        -0.00076926, -0.05410822],
       [ 0.01908831,  0.0095522 , -0.0474129 , ...,  0.03206478,
         0.03661669,  0.00331061]], dtype=float32)>

In [8]:
sent_1 = ["the location is great"]
sent_2 = ["amazing location"]


use(sent_1)


TypeError: 'AutoTrackable' object is not callable

In [60]:
import pandas as pd
from sklearn.model_selection import train_test_split
import keras

In [68]:
df = pd.read_csv("Hotel_Reviews.csv", parse_dates=['Review_Date'])


In [69]:
df = df.head(5000)

In [70]:
df["review"] = df["Negative_Review"] + df["Positive_Review"]
df["review_type"] = df["Reviewer_Score"].apply(
  lambda x: "bad" if x < 7 else "good"
)
df = df[["review", "review_type"]]

In [71]:
good_reviews = df[df["review_type"]=="good"]
bad_reviews = df[df["review_type"]=="bad"]

In [72]:
good_df = good_reviews.sample(n=len(bad_reviews), random_state=23)
bad_df = bad_reviews
review_df = good_df.append(bad_df).reset_index(drop=True)
print(review_df.shape)

(2070, 2)


In [73]:
from sklearn.preprocessing import OneHotEncoder
type_one_hot = OneHotEncoder(sparse=False).fit_transform(
  review_df.review_type.to_numpy().reshape(-1, 1)
)

In [74]:
train_reviews, test_reviews, y_train, y_test =\
  train_test_split(
    review_df.review,
    type_one_hot,
    test_size=.1,
    random_state=23
  )

In [75]:
from tqdm import tqdm
import tensorflow as tf

In [76]:
X_train = []
for r in tqdm(train_reviews):
  emb = embed(r)
  review_emb = tf.reshape(emb, [-1]).numpy()
  X_train.append(review_emb)
X_train = np.array(X_train)
X_test = []
for r in tqdm(test_reviews):
  emb = embed(r)
  review_emb = tf.reshape(emb, [-1]).numpy()
  X_test.append(review_emb)
X_test = np.array(X_test)
print(X_train.shape, y_train.shape)

100%|██████████| 207/207 [00:10<00:00, 18.94it/s]

(1863, 512) (1863, 2)


In [77]:
model = keras.Sequential()
model.add(keras.layers.Dense(
    units=256,
    input_shape=(X_train.shape[1], ),
    activation='relu'
  )
)
model.add(
  keras.layers.Dropout(rate=0.5)
)
model.add(
  keras.layers.Dense(
    units=128,
    activation='relu'
  )
)
model.add(
  keras.layers.Dropout(rate=0.5)
)
model.add(keras.layers.Dense(2, activation='softmax'))
model.compile(
    loss='categorical_crossentropy',
    optimizer=keras.optimizers.Adam(0.001),
    metrics=['accuracy']
)


In [78]:
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=16,
    validation_split=0.1,
    verbose=1,
    shuffle=True
)

Epoch 1/100
105/105 [==============================] - 1s 3ms/step - loss: 0.6526 - accuracy: 0.6147 - val_loss: 0.4727 - val_accuracy: 0.7701
Epoch 2/100
105/105 [==============================] - 0s 2ms/step - loss: 0.4701 - accuracy: 0.7881 - val_loss: 0.4292 - val_accuracy: 0.8021
Epoch 3/100
105/105 [==============================] - 0s 2ms/step - loss: 0.4403 - accuracy: 0.7899 - val_loss: 0.4051 - val_accuracy: 0.8396
Epoch 4/100
105/105 [==============================] - 0s 1ms/step - loss: 0.4052 - accuracy: 0.8192 - val_loss: 0.3939 - val_accuracy: 0.8556
Epoch 5/100
105/105 [==============================] - 0s 1ms/step - loss: 0.3618 - accuracy: 0.8422 - val_loss: 0.3975 - val_accuracy: 0.8449
Epoch 6/100
105/105 [==============================] - 0s 2ms/step - loss: 0.3489 - accuracy: 0.8511 - val_loss: 0.4125 - val_accuracy: 0.8289
Epoch 7/100
105/105 [==============================] - 0s 2ms/step - loss: 0.3236 - accuracy: 0.8692 - val_loss: 0.4361 - val_accuracy: 0.8289

Epoch 58/100
105/105 [==============================] - 0s 1ms/step - loss: 0.0198 - accuracy: 0.9920 - val_loss: 1.3107 - val_accuracy: 0.8021
Epoch 59/100
105/105 [==============================] - 0s 1ms/step - loss: 0.0267 - accuracy: 0.9913 - val_loss: 1.3399 - val_accuracy: 0.7807
Epoch 60/100
105/105 [==============================] - 0s 1ms/step - loss: 0.0194 - accuracy: 0.9948 - val_loss: 1.3987 - val_accuracy: 0.8075
Epoch 61/100
105/105 [==============================] - 0s 1ms/step - loss: 0.0286 - accuracy: 0.9856 - val_loss: 1.3336 - val_accuracy: 0.7914
Epoch 62/100
105/105 [==============================] - 0s 2ms/step - loss: 0.0401 - accuracy: 0.9925 - val_loss: 1.2589 - val_accuracy: 0.7968
Epoch 63/100
105/105 [==============================] - 0s 2ms/step - loss: 0.0289 - accuracy: 0.9929 - val_loss: 1.3054 - val_accuracy: 0.7807
Epoch 64/100
105/105 [==============================] - 0s 1ms/step - loss: 0.0338 - accuracy: 0.9857 - val_loss: 1.2945 - val_accuracy:

In [79]:
model.evaluate(X_test, y_test)


7/7 [==============================] - 0s 946us/step - loss: 1.5776 - accuracy: 0.8068


[1.5776022672653198, 0.8067632913589478]